In [1]:
import numpy as np
import pandas as pd


In [265]:
import csv
import numpy as np

cnt = 1
csv_file_path = 'crop_production.csv'
states = {}
districts = {}
season_to_number = {}
crops = {}
with open(csv_file_path, 'r') as file:
    csv_reader = csv.reader(file)
    for row in  csv_reader:
        cnt = cnt+1
        if cnt == 1 : continue
        states[row[0].strip()] = 1
        districts[row[1].strip()] = 1
        season_to_number[row[3].strip()] = 1
        crops[row[4].strip()] = 1
        
cnt = 1
for val in states.keys():
    states[val] = cnt
    cnt = cnt+1
cnt = 1
for val in districts.keys():
    districts[val] = cnt
    cnt = cnt+1
cnt = 1

for val in season_to_number.keys():
    season_to_number[val] = cnt
    cnt = cnt+1
cnt = 1
for val in crops.keys():
    crops[val] = cnt
    cnt = cnt+1



# New idea is to divide the crop produce by area so that we can predict for each district what yield would it give per area then
# multiply by area.

{'Season': 1, 'Kharif': 2, 'Whole Year': 3, 'Autumn': 4, 'Rabi': 5, 'Summer': 6, 'Winter': 7}


In [232]:
import csv
import numpy as np
from sklearn.model_selection import train_test_split


csv_file_path = 'crop_production.csv'

y = []
data = []
cnt = 0
with open(csv_file_path, 'r') as file:
    csv_reader = csv.reader(file)
    for row in csv_reader:
        cnt = cnt+1
        if cnt == 1 : continue
        try:
            val1 = float(row[6].strip())
            val2 = float(row[5])
            y.append(val1/val2)
        except Exception as e:
            # print(e)
            continue
        temp_row = row
        
        temp_row[1] = districts[temp_row[1].strip()]
        temp_row[3] = season_to_number[temp_row[3].strip()]
        temp_row[4] = crops[temp_row[4].strip()]
        
        temp1 = row[1:2]
        temp2 = row[3:5]
        
        data.append(temp1+temp2)
       
y = np.array(y)

data = np.array(data)
print(data.shape)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_normalized = scaler.fit_transform(data)
# i = 7
# print(data[i])
# print(y[i])


X_train, X_test, y_train, y_test = train_test_split(X_train_normalized, y, test_size = .15, random_state = 18)
X_train , X_cv , y_train , y_cv = train_test_split(X_train_normalized, y, test_size = .10, random_state = 18)

(242361, 3)


In [280]:
print(X_train)
# n_estimators=1000, max_depth=7, eta=0.1, subsample=0.7, colsample_bytree=0.8 

[[-1.36919046  1.0289498   1.98935297]
 [-1.13512263 -0.97610996  1.4156085 ]
 [-1.60834672 -0.30775671 -0.97499346]
 ...
 [-0.36168109 -0.97610996 -0.49687306]
 [-0.34132737 -0.97610996 -1.26186569]
 [ 1.04781434  1.0289498  -0.59249714]]


In [283]:
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV

param_grid_gb = {
    "n_estimators" : [1000],
    "eta" : [1e-3],
    "min_child_weight" : [55],
    "max_depth" : [7],
}

model = XGBRegressor()
mse_grid = GridSearchCV(estimator=model, param_grid=param_grid_gb, scoring='neg_mean_squared_error', cv=5, verbose=1, n_jobs=-1)

mse_grid.fit(X_train, y_train, eval_set=[(X_cv, y_cv)], early_stopping_rounds=20)

print("Best parameters:", mse_grid.best_params_)


# Procedure: First tune min_child_weight and max_depth
# Tune gamma
# Tune subsample and colsample
# Tune regularization (alpha paramater)
# Tune the estimators as required

# Previous run (27m 35.4s) Best parameter:  {'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 200, 'subsample': 1}
# Best parameter:  {'colsample_bytree': 1e-05, 'gamma': 0, 'learning_rate': 1e-07, 'max_depth': 1, 'min_child_weight': 1, 'n_estimators': 100, 'reg_alpha': 1e-07, 'subsample': 1}


Fitting 5 folds for each of 3 candidates, totalling 15 fits
[0]	validation_0-rmse:1084.01091
[1]	validation_0-rmse:1083.78960
[2]	validation_0-rmse:1083.56871
[3]	validation_0-rmse:1083.34811
[4]	validation_0-rmse:1083.12793
[5]	validation_0-rmse:1082.90807
[6]	validation_0-rmse:1082.68857
[7]	validation_0-rmse:1082.46948
[8]	validation_0-rmse:1082.25075
[9]	validation_0-rmse:1082.03235
[10]	validation_0-rmse:1081.81433
[11]	validation_0-rmse:1081.59666
[12]	validation_0-rmse:1081.37939
[13]	validation_0-rmse:1081.16246
[14]	validation_0-rmse:1080.94586
[15]	validation_0-rmse:1080.72964
[16]	validation_0-rmse:1080.51385
[17]	validation_0-rmse:1080.29833
[18]	validation_0-rmse:1080.08318
[19]	validation_0-rmse:1079.86841
[20]	validation_0-rmse:1079.65403
[21]	validation_0-rmse:1079.43992
[22]	validation_0-rmse:1079.22621
[23]	validation_0-rmse:1079.01289
[24]	validation_0-rmse:1078.79990
[25]	validation_0-rmse:1078.58726
[26]	validation_0-rmse:1078.37500
[27]	validation_0-rmse:1078.1630

In [6]:
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
# from sklearn.model_selection import RandomizedSearchCV

# param_random_gb = {
#               'learning_rate': np.arange(0.05,0.15, 0.05),
#               # 'n_estimators' : np.arange(200,1000,100)
#             #   'subsample' : np.arange(0.9,1, 0.05),
#             #   "colsample_bytree" : np.arange(0.9,1,0.05)
#               }

# # Regressor Instantiation
# gb = XGBRegressor()

# # Here we specified a total of 25 iterations
# mse_random = RandomizedSearchCV(estimator = gb, param_distributions= param_random_gb, n_iter = 25, 
#                                 scoring = 'neg_mean_squared_error', cv = 4, verbose = 1)

# mse_random.fit(X_train_normalized1,y_train1)

# print("Best parameter: ", mse_random.best_params_)

In [284]:
# Best parameter:  {'colsample_bytree': 1, 'gamma': 1e-05, 'learning_rate': 1e-07, 'max_depth': 4, 'min_child_weight': 12, 'n_estimators': 100, 'reg_alpha': 1e-08, 'subsample': 0.9}


model = XGBRegressor(n_estimators=1000,eta = 1e-03,min_child_weight = 55)
# eval_set = [(X_test,y_test)]
model.fit   (X_train,y_train,eval_set=[(X_train,y_train),(X_test,y_test)],early_stopping_rounds=1000,)

# print("Best parameter: ", mse_grid.best_params_)

[0]	validation_0-rmse:783.26057	validation_1-rmse:944.14687
[1]	validation_0-rmse:782.97005	validation_1-rmse:943.90200
[2]	validation_0-rmse:782.68003	validation_1-rmse:943.65760
[3]	validation_0-rmse:782.39044	validation_1-rmse:943.41354
[4]	validation_0-rmse:782.10136	validation_1-rmse:943.16991
[5]	validation_0-rmse:781.81271	validation_1-rmse:942.92670
[6]	validation_0-rmse:781.52454	validation_1-rmse:942.68386
[7]	validation_0-rmse:781.23681	validation_1-rmse:942.44142
[8]	validation_0-rmse:780.94959	validation_1-rmse:942.19941
[9]	validation_0-rmse:780.66280	validation_1-rmse:941.95780
[10]	validation_0-rmse:780.37650	validation_1-rmse:941.71661
[11]	validation_0-rmse:780.09064	validation_1-rmse:941.47577
[12]	validation_0-rmse:779.80525	validation_1-rmse:941.23537
[13]	validation_0-rmse:779.52033	validation_1-rmse:940.99536
[14]	validation_0-rmse:779.23585	validation_1-rmse:940.75573
[15]	validation_0-rmse:778.95185	validation_1-rmse:940.51655
[16]	validation_0-rmse:778.66831	v

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eta=0.001, eval_metric=None,
             feature_types=None, gamma=None, gpu_id=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=None, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
             max_leaves=None, min_child_weight=55, missing=nan,
             monotone_constraints=None, n_estimators=1000, n_jobs=None,
             num_parallel_tree=None, predictor=None, ...)

In [285]:
# training_error = 0
from sklearn.metrics import mean_squared_error
print("Training error:")
print(mean_squared_error(y_train,model.predict(X_train)))
print("Test error:")
print(mean_squared_error(y_test,model.predict(X_test)))
# Previously
# Training error:
# 2041076666.4211206
# Test error:
# 71631908865818.11
# Overfit
# Now
# Training error:
# 444956956552237.3
# Test error:
# 61499447426576.72
# Ok

Training error:
414609.48835546925
Test error:
684110.7743953344


In [310]:
# This will be used for user inputs
# State_Name = states['Andaman and Nicobar Islands']
District_Name = districts['NICOBARS']
# Crop_Year = 2000
Season = season_to_number['Winter']
Crop = crops['Coconut']
Area = 18168.00
# print(State_Name,District_Name,Crop_Year,Season,Crop,Area)
X_predict = np.zeros((1,3))
X_predict[0] = np.array([District_Name,Season,Crop])
X_pred_normalized = scaler.transform(X_predict)

Production = model.predict(X_pred_normalized)
print(X_pred_normalized)
print(Production*Area)

[[-1.65923103  2.36565631 -1.07061753]]
[92153376.]


In [11]:
import pickle
filename = "finalized_model.sav"
# pickle.dump(model, open(filename, 'wb'))
 
# some time later...
 
# load the model from disk
model = pickle.load(open(filename, 'rb'))
# result = loaded_model.score(X_test, Y_test)

In [84]:
import pickle
filename = "finalized_model2.sav"
pickle.dump(model, open(filename, 'wb'))
 
# some time later...
#  Does not perform well on small aread like 1 10 100
# load the model from disk
# model = pickle.load(open(filename, 'rb'))
# result = loaded_model.score(X_test, Y_test)

In [271]:
# import pickle
filename = "finalized_model3.sav"
# pickle.dump(model, open(filename, 'wb'))
 
# some time later...
#  Does not perform well on small aread like 1 10 100
# load the model from disk
model = pickle.load(open(filename, 'rb'))
# result = loaded_model.score(X_test, Y_test)

In [299]:
import pickle
filename = "finalized_model4.sav"
pickle.dump(model, open(filename, 'wb'))
 
# some time later...
#  Does not perform well on small aread like 1 10 100
# load the model from disk
# model = pickle.load(open(filename, 'rb'))
# result = loaded_model.score(X_test, Y_test)